# Problem 1

The hinge loss penalty is defined as 
$$\ell_1(h, (x_i,y_i)) = \max_{y\in \mathcal{Y}\backslash \{y_i\}}\left(\Delta(y_i,y)-m_{i,y}(h)\right)_+ $$
while the generalized hinge loss is defined as 
$$\ell_2(h, (x_i,y_i)) = \max_{y\in \mathcal{Y}}\left[\Delta(y_i,y)+h(x_i,y)-h(x_i,y_i)\right]$$

## 1.1

Supposing $\Delta(y,y) = 0$ then 
\begin{align*}
\ell_2(h, (x_i,y_i)) &= \max_{y\in \mathcal{Y}}\left[\Delta(y_i,y)+h(x_i,y)-h(x_i,y_i)\right]\\
&= \max \left(\Delta(y_i,y_i) + h(x_i,y_i) - h(x_i,y_i), \max_{y\in \mathcal{Y}\backslash\{y_i\}}\left[\Delta(y_i,y)+h(x_i,y)-h(x_i,y_i)\right]\right)\\
&= \max \left(0, \max_{y\in \mathcal{Y}\backslash\{y_i\}}\left[\Delta(y_i,y)+h(x_i,y)-h(x_i,y_i)\right]\right)\\
&= \max \left(0, \max_{y\in \mathcal{Y}\backslash\{y_i\}}\left[\Delta(y_i,y)-m_{i,y}(h)\right]\right)\\
&= \max_{y\in \mathcal{Y}\backslash\{y_i\}}\left[\Delta(y_i,y)-m_{i,y}(h)\right]_+\\
\end{align*}

## 1.2

Supposing $m_{i,y}(h) = h(x_i,y_i)-h(x_i,y) \ge \Delta(y_i,y)$ then $\Delta(y_i,y) -m_{i,y}(h) \le 0$ which implies $\ell_1(h,(x_i,y_i))=0$. This further implies $\ell_1(h,(x_i,y_i))=0$

Under the above conditions, since $h(x_i,y_i)-h(x_i,y)\gt 0$ then $h(x_i,y_i)\gt h(x_i,y)$ we must have that $y_i = \arg \max h(x_i,y)$ which means we make a correct prediction.

# Problem 2

The $\ell_2$ regularized loss function is given by 
$$J(w) = \lambda \|w\|^2 + \frac{1}{n} \sum_{i=1}^n \max_{y\in \mathcal{Y}}\left[\Delta(y_i,y) + \langle w, \Psi(x_i,y)-\Psi(x_i,y_i)\rangle\right]$$

## Problem 2.1

The function $J(w)$ is convex since the first term is the sum of squares and the second terms is the pointwise maximum of an affine function of $w$. Since affine functions are convex, the pointwise maximum of convex functions is convex, and the sum of convex functions is convex, then $J(w)$ is convex.

## Problem 2.2

A subgradient of $J(w)$ is given by 
$$2\lambda w + 1/n\sum_{i=1}^n \Psi(x_i,\hat{y}_i) - \Psi(x_i,y_i)$$ where $\hat{y}_i = \arg \max_{y\in \mathcal{Y}}\left[\Delta(y_i,y) + \langle w, \Psi(x_i,y)-\Psi(x_i,y_i)\rangle\right]$

# Problem 3

Given $\mathcal{Y} = \{-1,1\}$ and $\Delta(y,\hat{y})=\mathbf{1}(y\ne\hat{y})$ as well as the compatibility score functions $h(x,1) = g(x)/2$ and $h(x,-1)=-g(x)/2$ the multiclass hinge loss is given by 

\begin{align*}
\ell(h,(x,y)) &= \max_{y'\in\mathcal{Y}}\left[\Delta(y,y') + h(x,y')-h(x,y)\right]\\
&= \max\left(\Delta(y,-1) + h(x,-1)-h(x,y), \Delta(y,1) + h(x,1)-h(x,y)\right)\\
&= \max\left(\Delta(y,-1) -g(x)/2-yg(x)/2, \Delta(y,1) + g(x)/2-yg(x)/2\right)\\
&= \max\left\{0,1-yg(x)\right\}\\
\end{align*}
